In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

In [3]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [4]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [5]:
feat_dim = train_data.shape[0]

In [6]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_master.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_master.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    #np.savetxt('NN_ans.txt',np.array(a),fmt='%f')
    #print(a)
    a = pd.DataFrame(a)
    a.to_csv('NN_base.csv',header=None)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [ ]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.05701204721863
epoch:1
error:36.92651651439198
epoch:2
error:36.80728492884864
epoch:3
error:36.67601370472186
epoch:4
error:36.53315511804657
epoch:5
error:36.39624939359638
epoch:6
error:36.250520496442434
epoch:7
error:36.09061186125485
epoch:8
error:35.908539173674
epoch:9
error:35.7190051948577
epoch:10
error:35.522237326164294
epoch:11
error:35.301726228978005
epoch:12
error:35.07751859637775
epoch:13
error:34.89042033877749
epoch:14
error:34.609097065006466
epoch:15
error:34.330295419631305
epoch:16
error:33.98562065410737
epoch:17
error:33.634576123868044
epoch:18
error:33.27478674805951
epoch:19
error:32.909474045924966
epoch:20
error:32.522441987386806
epoch:21
error:32.12386299320828
epoch:22
error:31.654289294954722
epoch:23
error:31.29492692836352
epoch:24
error:30.77212867884864
epoch:25
error:30.268780825517464
epoch:26
error:29.667256225743856
epoch:27
error:29.311870856241914
epoch:28
error:28.702902652005175
epoch:29
error:28.17841910575679
epoch:30
e

epoch:241
error:1.7766039059973318
epoch:242
error:1.746740580530401
epoch:243
error:1.70918745704641
epoch:244
error:1.7028017426524094
epoch:245
error:1.7720235057204075
epoch:246
error:1.8032722423795278
epoch:247
error:1.7933036932507276
epoch:248
error:1.7279959181456177
epoch:249
error:1.7558481628496927
epoch:250
error:1.9018199372877587
epoch:251
error:1.6941218012208927
epoch:252
error:1.8121347365681597
epoch:253
error:1.7178568179980596
epoch:254
error:1.6896173593143597
epoch:255
error:1.6467199880740622
epoch:256
error:1.7558454782503234
epoch:257
error:1.6538307022153946
epoch:258
error:1.6927457071676908
epoch:259
error:1.7364734092011642
epoch:260
error:1.6885713130659767
epoch:261
error:1.7196948210603977
epoch:262
error:1.697664524882762
epoch:263
error:1.7303203074466365
epoch:264
error:1.6724953003719276
epoch:265
error:1.6375942452296248
epoch:266
error:1.6706555602158797
epoch:267
error:1.6963124343062743
epoch:268
error:1.7196191785252264
epoch:269
error:1.731309

epoch:477
error:1.589770476228978
epoch:478
error:1.5566910007175776
epoch:479
error:1.6468120540406694
epoch:480
error:1.5160387441380982
epoch:481
error:1.7598943278521184
epoch:482
error:1.8049231130740622
epoch:483
error:1.6547658903015847
epoch:484
error:1.4900404521749677
epoch:485
error:1.4767843742419955
epoch:486
error:1.4836027828872898
epoch:487
error:1.5501325244582795
epoch:488
error:1.6287538595063875
epoch:489
error:1.5840478578792043
epoch:490
error:1.5529095056294469
epoch:491
error:1.590104314056436
epoch:492
error:1.54235808260228
epoch:493
error:1.5055650165750323
epoch:494
error:1.4617539340434993
epoch:495
error:1.4820063937681922
epoch:496
error:1.4471996154390363
epoch:497
error:1.6567520200820667
epoch:498
error:1.625472015736174
epoch:499
error:1.6481085576184509
epoch:500
error:1.5397477045096215
epoch:501
error:1.5515736805667852
epoch:502
error:1.471607045399418
epoch:503
error:1.5721359746422219
epoch:504
error:1.4814791068685318
epoch:505
error:1.46293436

epoch:713
error:1.424295561125485
epoch:714
error:1.429772933426989
epoch:715
error:1.4391347630983182
epoch:716
error:1.3721726431112549
epoch:717
error:1.4108659317391656
epoch:718
error:1.4936218656512774
epoch:719
error:1.7119216425452781
epoch:720
error:1.5069578498948901
epoch:721
error:1.5836363245876455
epoch:722
error:1.4538631069089587
epoch:723
error:1.453914114296976
epoch:724
error:1.4088717481605757
epoch:725
error:1.4524222666356728
epoch:726
error:1.460677251778784
epoch:727
error:1.4303543860668662
epoch:728
error:1.3858770491389067
epoch:729
error:1.4726451957673028
epoch:730
error:1.4142093633772639
epoch:731
error:1.36189757590152
epoch:732
error:1.4126118688348965
epoch:733
error:1.4402875616510349
epoch:734
error:1.4082944013785577
epoch:735
error:1.4534222009419469
epoch:736
error:1.4469940067108666
epoch:737
error:1.563594684872251
epoch:738
error:1.475720167468467
epoch:739
error:1.386401651439198
epoch:740
error:1.366304740560317
epoch:741
error:1.440540545662

epoch:949
error:1.4219214277773287
epoch:950
error:1.3523899882761967
epoch:951
error:1.420650822687581
epoch:952
error:1.3795987187196799
epoch:953
error:1.3065855117541236
epoch:954
error:1.355977244704075
epoch:955
error:1.3696807032260672
epoch:956
error:1.38285324274337
epoch:957
error:1.4025286715212646
epoch:958
error:1.3949108839343467
epoch:959
error:1.3249904933598804
epoch:960
error:1.3290762956824063
epoch:961
error:1.3908212915891818
epoch:962
error:1.3628273947889715
epoch:963
error:1.3868659292124839
epoch:964
error:1.3709220935579722
epoch:965
error:1.419828387774903
epoch:966
error:1.3770677731848318
epoch:967
error:1.2907074487841608
epoch:968
error:1.2940087953792043
epoch:969
error:1.4303847062479786
epoch:970
error:1.315556337423189
epoch:971
error:1.4858711114367724
epoch:972
error:1.3912651979907826
epoch:973
error:1.3772845940633085
epoch:974
error:1.3411647308073253
epoch:975
error:1.3196953579802717
epoch:976
error:1.2964051951103654
epoch:977
error:1.32800987

...

epoch:24881
error:1.074507423391009
epoch:24882
error:1.0478308752930952
epoch:24883
error:1.042253067391656
epoch:24884
error:1.061653877445828
epoch:24885
error:1.050568929773407
epoch:24886
error:1.0602393304546005
epoch:24887
error:1.058707766514392
epoch:24888
error:1.0828821100824708
epoch:24889
error:1.0522198794267466
epoch:24890
error:1.0593491488872493
epoch:24891
error:1.0581999034807568
epoch:24892
error:1.0546368084472024
epoch:24893
error:1.0675990017080368
epoch:24894
error:1.1179229223095892
epoch:24895
error:1.099056347529916
epoch:24896
error:1.0860938384338616
epoch:24897
error:1.191316473838737
epoch:24898
error:1.1598638655451567
epoch:24899
error:1.1525814160029917
epoch:24900
error:1.1058603067138988
epoch:24901
error:1.113000156654269
epoch:24902
error:1.0955259414416236
epoch:24903
error:1.1015864245179092
epoch:24904
error:1.0884698667933377
epoch:24905
error:1.109838804020456
epoch:24906
error:1.1201733272103411
epoch:24907
error:1.0980041424947444
epoch:2490

epoch:25104
error:1.0413931270466121
epoch:25105
error:1.0561243923330368
epoch:25106
error:1.0534799040366267
epoch:25107
error:1.0541818478129044
epoch:25108
error:1.076700977952175
epoch:25109
error:1.066114102421572
epoch:25110
error:1.0469326399276357
epoch:25111
error:1.133602167008611
epoch:25112
error:1.1002170419631307
epoch:25113
error:1.0771912331722995
epoch:25114
error:1.0648048075173835
epoch:25115
error:1.126842898508247
epoch:25116
error:1.085994666174806
epoch:25117
error:1.1349471512926503
epoch:25118
error:1.1166349462827458
epoch:25119
error:1.0761532407220247
epoch:25120
error:1.0609879388846215
epoch:25121
error:1.0659487626839423
epoch:25122
error:1.0670028627304333
epoch:25123
error:1.0740289330328268
epoch:25124
error:1.0576138712301908
epoch:25125
error:1.0474641116439602
epoch:25126
error:1.0658219548431436
epoch:25127
error:1.0515512562661709
epoch:25128
error:1.0686139381872575
epoch:25129
error:1.0588374958309752
epoch:25130
error:1.0672276584482132
epoch:

epoch:25328
error:1.0797703435781856
epoch:25329
error:1.0453075098035252
epoch:25330
error:1.0534857469881953
epoch:25331
error:1.085709309053606
epoch:25332
error:1.0966101247928122
epoch:25333
error:1.1370153982303122
epoch:25334
error:1.0911915768010187
epoch:25335
error:1.070218696939683
epoch:25336
error:1.0567500619037031
epoch:25337
error:1.0369434035919307
epoch:25338
error:1.0460835169388745
epoch:25339
error:1.1239365828145214
epoch:25340
error:1.1037387625828752
epoch:25341
error:1.0932888805789134
epoch:25342
error:1.0950575578104786
epoch:25343
error:1.0516414272214587
epoch:25344
error:1.0706635508469438
epoch:25345
error:1.0794038957642706
epoch:25346
error:1.0424205390170198
epoch:25347
error:1.0643844308396668
epoch:25348
error:1.0572842971782017
epoch:25349
error:1.0548649204347913
epoch:25350
error:1.0844304922481403
epoch:25351
error:1.0801733493188066
epoch:25352
error:1.0416683774407747
epoch:25353
error:1.0597633667781776
epoch:25354
error:1.068419857444615
epoc

epoch:25551
error:1.0498075300169794
epoch:25552
error:1.0606752620168984
epoch:25553
error:1.1147932321565734
epoch:25554
error:1.0679943484445746
epoch:25555
error:1.0977026777773287
epoch:25556
error:1.0812388194332148
epoch:25557
error:1.074230988614772
epoch:25558
error:1.0705981729564198
epoch:25559
error:1.093041660560519
epoch:25560
error:1.0503146824213696
epoch:25561
error:1.055115298805385
epoch:25562
error:1.04797331697728
epoch:25563
error:1.0459153346842658
epoch:25564
error:1.079106221069494
epoch:25565
error:1.0351423532503234
epoch:25566
error:1.0292471309528621
epoch:25567
error:1.0344323556759378
epoch:25568
error:1.0529592496765847
epoch:25569
error:1.127274566295076
epoch:25570
error:1.0979864557224288
epoch:25571
error:1.0765017648872088
epoch:25572
error:1.0747104264786143
epoch:25573
error:1.0680981792731241
epoch:25574
error:1.064570931537031
epoch:25575
error:1.0617167286545925
epoch:25576
error:1.0560291680142302
epoch:25577
error:1.050228064612306
epoch:2557

epoch:25775
error:1.108890824607859
epoch:25776
error:1.1469661816279917
epoch:25777
error:1.097327070742036
epoch:25778
error:1.1088205512714262
epoch:25779
error:1.1018383820646023
epoch:25780
error:1.099096616520456
epoch:25781
error:1.0915233616995472
epoch:25782
error:1.1458427557507276
epoch:25783
error:1.1508406901883894
epoch:25784
error:1.1421980173128234
epoch:25785
error:1.1673151290123707
epoch:25786
error:1.160368096473763
epoch:25787
error:1.1108820077518597
epoch:25788
error:1.097558814834654
epoch:25789
error:1.1138640449395618
epoch:25790
error:1.1461987020435802
epoch:25791
error:1.1134069524175292
epoch:25792
error:1.1452600397699708
epoch:25793
error:1.1261649582086837
epoch:25794
error:1.1165586720771346
epoch:25795
error:1.084375063167044
epoch:25796
error:1.0778256671703186
epoch:25797
error:1.0760667808305708
epoch:25798
error:1.0582505160747493
epoch:25799
error:1.0821458981848318
epoch:25800
error:1.064729086023407
epoch:25801
error:1.083227870689481
epoch:258

In [ ]:
y_valid = valid_labels#.cuda()
x_valid = valid_features.cuda()
net = torch.load('.//model_NN_master.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)



In [ ]:
predict(x_test,use_gpu)


In [ ]:
from graphviz import Digraph
from torchviz import make_dot

In [19]:
make_dot(net(torch.randn(30,1)))


RuntimeError: size mismatch, m1: [30 x 1], m2: [30 x 30] at /opt/conda/conda-bld/pytorch_1549628766161/work/aten/src/TH/generic/THTensorMath.cpp:940